In [1]:
%load_ext autoreload
%autoreload 2

# Non-adiabatic dynamics 
This tutorial shows how to run non-adiabatic dynamics with a trained model using the [Zhu-Nakamura surface hopping method](https://pubs.rsc.org/en/content/articlelanding/2014/cp/c4cp03498h).

First let's import dependencies:


In [23]:
import sys
import os

# so that NFF is in your path
sys.path.insert(0, "..")

import json
import numpy as np
from nff.md.zhu_nakamura.dynamics import CombinedZhuNakamura
from ase.io.trajectory import Trajectory
import nglview as nv
from nff.md.utils import csv_read
from ase import Atoms



Now we'll find a trained model. The trained azobenzene models can be found in `NeuralForceField/models/azo_derivatives`. The sub-folders are for diabatic and adiabatic models, trained either with the full set of geometries, or with 40 species held out. There are also three models trained with different splits and different initialization from random seeds:

In [3]:
print(os.listdir('../models/azo_derivatives'))
print(os.listdir('../models/azo_derivatives/all_diabatic'))

['all_diabatic', 'holdout_diabatic', 'holdout_adiabatic']
['seed_0', 'seed_2', 'seed_1']


We'll use the diabatic model trained on all species, with seed 0: `../models/azo_derivatives/all_diabatic/seed_0`.

## ZN

The script for Tully's surface hopping is `NeuralForceField/nff/md/zhu_nakamura/run_zn`. If you run the script and supply the path of a JSON parameter file, it will do the rest for you. Here we'll go through some parameters to give, and show a little of what goes on behind the scenes.

We'll have to define `ground_params`, `zhu_params`, `combined_params`, and `all_params`. The first is for parameters in the ground state MD simulation, the second for ZN surface hopping, and the third is for batching. The fourth is for some remaining parameters, which we'll explain below.

Let's define `ground_params`:

In [4]:
ground_params = {'ttime': 50,
                 'logfile': 'ground.log',
                 'max_time': 200,
                 'savefile': 'ground.trj',
                 'timestep': 0.5,
                 'equil_time': 100,
                 'thermostat': 'nosehoover',
                 'loginterval': 10,
                 'temperature': 300}

Now let's do `zhu_params` and `batched_params`:

In [5]:
zhu_params = {'log_file': 'trj.log',
 'max_time': 200,
 'out_file': 'trj.csv',
 'timestep': 0.5,
 'num_states': 2,
 'max_gap_hop': 0.021673306772908366,
 'save_period': 5,
 'initial_surf': 1,
 'initial_time': 0.0}

batched_params = {'cutoff': 5.0,
 'device': 1,
 'num_trj': 10,
 'batch_size': 5,
 'nbr_update_period': 10}

Lastly we'll define `all_params`, which has the starting coordinates and the model path:

In [6]:
with open('data/azo_coords.json', 'r') as f:
    coords = json.load(f)

all_params = {"coords": coords, # starting geometry of the molecule
              'model_path': '../models/azo_derivatives/all_diabatic/seed_0',
              "cutoff": 5.0,
              "zhu_params": zhu_params,
              "ground_params": ground_params,
              "batched_params": batched_params
             }

When we run the script from the command line, it parses these three dictionaries from a file and makes an instance of `CombinedZhuNakamura`, like this:

In [7]:
from nff.md.zhu_nakamura.run_zn import coords_to_xyz, make_dataset, make_trj

coords = all_params["coords"]
nxyz = [coords_to_xyz(coords)]


print('loading models')

dataset = make_dataset(nxyz=nxyz, all_params=all_params)

print('running ground state + Zhu-Nakamura dynamics')

zn = make_trj(all_params=all_params,
              dataset=dataset)



loading models
running ground state + Zhu-Nakamura dynamics


/home/saxelrod/miniconda3/envs/nff/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Now we can run it!

In [8]:
zn.run()


../nff/nn/modules/diabat.py:140: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2500.)
  ad_energies, u = torch.symeig(d_mat, True)


Completed step 0
Completed step 10
Completed step 20
Completed step 30
Completed step 40
Completed step 50
Completed step 60
Completed step 70
Completed step 80
Completed step 90
Completed step 100
Completed step 110
Completed step 120
Completed step 130
Completed step 140
Completed step 150
Completed step 160
Completed step 170
Completed step 180
Completed step 190
Completed step 200
Completed step 210
Completed step 220
Completed step 230
Completed step 240
Completed step 250
Completed step 260
Completed step 270
Completed step 280
Completed step 290
Completed step 300
Completed step 310
Completed step 320
Completed step 330
Completed step 340
Completed step 350
Completed step 360
Completed step 370
Completed step 380
Completed step 390
Completed step 400
Neural ZN terminated normally.


We can view the ground-state log file:

In [9]:
with open('ground.log', 'r') as f:
    ground_log = f.read()
print(ground_log)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           2.0793      -0.8128       2.8921   658.1
0.0050           2.0460       0.6323       1.4137   321.7
0.0100           2.0074       0.6587       1.3487   306.9
0.0150           1.9685       0.6079       1.3606   309.6
0.0200           1.9191       0.3891       1.5299   348.1
0.0250           1.8746       0.7113       1.1633   264.7
0.0300           1.8280       0.4594       1.3687   311.4
0.0350           1.7805       0.5009       1.2796   291.2
0.0400           1.7348       0.6183       1.1165   254.0
0.0450           1.7018       0.5671       1.1348   258.2
0.0500           1.6612       0.1513       1.5099   343.6
0.0550           1.6148       0.3042       1.3106   298.2
0.0600           1.5646       0.1755       1.3891   316.1
0.0650           1.5132       0.2880       1.2253   278.8
0.0700           1.4709       0.4620       1.0089   229.6
0.0750           1.4407       0.3959       1.0448   237.7
0.0800         

We see that all energies fluctuate, as kinetic energy is being added into the system fo the thermostat. The temperature also varies, and over enough time it will average out to 300 K. 

To get the actual geometries, energies, and forces, we can load the trajectory file. And we can visualize it with `nglview`:



In [10]:
trj = Trajectory('ground.trj')
nv.show_asetraj(trj)

NGLWidget(max_frame=40)

Unlike neural Tully, neural ZN saves the trajectories separately from each other. This may be changed in the future, since saving in one file is much easier.

In any case we can examine individual trajectories:

In [13]:
with open('trj_0.log', 'r') as f:
    zn_log = f.read()
print(zn_log)

ZHU-NAKAMURA DYNAMICS:  Completed step 1. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 3.2621356678776063 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 11. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 2.848301627555715 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 21. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 2.1683965365997238 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 31. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 1.53168961295214 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 40. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 0.6689230833772156 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 50. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 0.5954116252746224 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 61. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 0.7876349541774498 eV
ZHU-NAKAMURA DYNAMICS:  Completed step

To get the geometries, forces, etc., we can load the trajectory's CSV file:



In [35]:
trj_dics = csv_read('trj_4.csv')
nxyz_list = [np.array(i['nxyz']) for i in trj_dics]
trj = [Atoms(numbers=nxyz[:, 0], positions=nxyz[:, 1:])
      for nxyz in nxyz_list]

The output contains `state_dicts`, a set of dictionaries with information about each trajectory for each time step, and `trjs`, a set of ASE trajectories. We can visualize one of the trajectories:

In [36]:
nv.show_asetraj(trj)

NGLWidget(max_frame=40)